In [15]:
import pandas as pd
import glob

In [16]:
user_infos = pd.concat([pd.read_csv(fpath) for fpath in glob.glob('../data/files/treino/*.csv')])

In [60]:
user_historys = user_infos[['userId', 'history','timestampHistory','numberOfClicksHistory','timeOnPageHistory','scrollPercentageHistory','pageVisitsCountHistory','timestampHistory_new']]
user_historys = user_historys.set_index('userId').apply(lambda row: row.str.split(','), axis=1)
user_historys = user_historys.apply(pd.Series.explode).reset_index()
user_historys['numberOfClicksHistory'] = user_historys['numberOfClicksHistory'].astype(int)
user_historys.head()

,userId,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory,timestampHistory_new
0,e5f68d5e7cdbe56d6984589b4baa6ebfc5e8a8a918e57d...,50028008-aa11-4519-9d75-452c84dd27fb,1657264136826,4,10081,16.34,1,1657264136826
1,e5f68d5e7cdbe56d6984589b4baa6ebfc5e8a8a918e57d...,d031af1b-f939-47c1-a589-e6d691b66d91,1657552410329,41,70000,79.35,1,1657552410329
2,e5f68d5e7cdbe56d6984589b4baa6ebfc5e8a8a918e57d...,bf257382-74fb-4392-ad6a-143240e39f81,1657552529759,6,12216,13.36,1,1657552529759
3,e5f68d5e7cdbe56d6984589b4baa6ebfc5e8a8a918e57d...,66a9efac-fd43-4fd1-9824-c404b08efa5d,1657632476730,0,90000,37.95,1,1657632476730
4,e5f68d5e7cdbe56d6984589b4baa6ebfc5e8a8a918e57d...,7a349b09-badc-40a9-a194-83d959aeb50c,1657632495643,0,161291,73.83,1,1657632495643


In [52]:
news_item = pd.concat([pd.read_csv(fpath) for fpath in glob.glob('../data/itens/itens/*.csv')])
news_item.head()

,page,url,issued,modified,title,body,caption
0,13db0ab1-eea2-4603-84c4-f40a876c7400,http://g1.globo.com/am/amazonas/noticia/2022/0...,2022-06-18 20:37:45+00:00,2023-04-15 00:02:08+00:00,Caso Bruno e Dom: 3º suspeito tem prisão tempo...,"Após audiência de custódia, a Justiça do Amazo...",Jeferson da Silva Lima foi escoltado por agent...
1,92907b73-5cd3-4184-8d8c-e206aed2bf1c,http://g1.globo.com/pa/santarem-regiao/noticia...,2019-06-20 17:19:52+00:00,2023-06-16 20:19:15+00:00,Linguajar dos santarenos é diferenciado e chei...,Vista aérea de Santarém\nÁdrio Denner/ AD Prod...,As expressões santarenas não significam apenas...
2,61e07f64-cddf-46f2-b50c-ea0a39c22050,http://g1.globo.com/mundo/noticia/2022/07/08/e...,2022-07-08 08:55:52+00:00,2023-04-15 04:25:39+00:00,Ex-premiê Shinzo Abe morre após ser baleado no...,Novo vídeo mostra que assassino de Shinzo Abe ...,Ex-primeiro-ministro foi atingido por tiros de...
3,30e2e6c5-554a-48ed-a35f-6c6691c8ac9b,http://g1.globo.com/politica/noticia/2021/09/0...,2021-09-09 19:06:46+00:00,2023-06-07 17:44:54+00:00,"Relator no STF, Fachin vota contra marco tempo...","Relator no STF, Fachin vota contra marco tempo...",Ministro defendeu que posse indígena é diferen...
4,9dff71eb-b681-40c7-ac8d-68017ac36675,http://g1.globo.com/politica/noticia/2021/09/1...,2021-09-15 19:16:13+00:00,2023-06-07 17:43:39+00:00,"\nApós 2 votos, pedido de vista suspende julga...",Após um pedido de vista (mais tempo para análi...,"Pelo marco temporal, índios só podem reivindic..."


In [61]:
user_historys.query('history == "1db21691-869e-4391-afc9-9482f367b6bb"')['numberOfClicksHistory'].max()


np.int64(111)

In [65]:
user_historys.groupby('history')[['numberOfClicksHistory']].sum().sort_values('numberOfClicksHistory',ascending=False).head(5)

,numberOfClicksHistory
history,
d2593c3d-2347-40d9-948c-b6065e8459a9,316424
f0a78e58-ec7e-494c-9462-fbd6446a9a89,279037
f6b5d170-48b9-4f8e-88d4-c84b6668f3bd,258710
a36c98b5-f159-48f8-9f5a-1fc6ea9956c8,244181
1f32787b-de2b-49be-8c20-ddaeae34cc22,229747


In [63]:
news_item.set_index('page').loc['d2593c3d-2347-40d9-948c-b6065e8459a9'].head()

url         http://g1.globo.com/rj/rio-de-janeiro/noticia/...
issued                              2022-07-11 03:20:57+00:00
modified                            2022-07-12 23:06:28+00:00
title       Anestesista é preso em flagrante por estupro d...
body        Vídeo mostra momento em que anestesista estupr...
Name: d2593c3d-2347-40d9-948c-b6065e8459a9, dtype: object